# Notebook: Convert to Formatted JSON

## Packages

In [92]:
import sys
import os
sys.path.append(os.path.abspath('../03 dataset split/'))
from format_labelstudio_json import format_json

In [93]:
import pandas as pd
import json

## Settings

In [94]:
MODEL = "Llama13B"
FEW_SHOT = "random"
N_EXAMPLES = 25

## Code

In [95]:
with open(f"annotation_datasets/synth_annotation_labelstudio_output/annotation_{MODEL}_{FEW_SHOT}_{N_EXAMPLES}.json", 'r') as json_file:
    synthetic_data = json.load(json_file)    

In [96]:
synth_llm_labels = []
for split_id in range(5):
    with open(f"../07 train classifier/synth/{MODEL}/{FEW_SHOT}/split_{split_id}.json", 'r') as json_file:
       synthetic_data_split = json.load(json_file)
       for example in  synthetic_data_split:
           synth_llm_labels.append(example)    

In [97]:
# ground truth - annotations
synth_annotated = format_json(synthetic_data)
synth_annotated_ids = [example["id"] for example in synth_annotated]

# label from llm
synth_llm_labels = [example for example in synth_llm_labels if example["id"] in synth_annotated_ids]
def get_index(example):
    return synth_annotated_ids.index(example["id"])
synth_llm_labels = sorted(synth_llm_labels, key=get_index)

In [98]:
len(synth_llm_labels)

25

In [106]:
synth_annotated[1]["tags"], synth_annotated[1]["text"]

([], 'Wir würden wiederkommen.')

In [107]:
synth_llm_labels[1]["tags"], synth_llm_labels[1]["text"]

([{'text': None,
   'start': 0,
   'end': 0,
   'tag_with_polarity': 'GENERAL-IMPRESSION-NEUTRAL',
   'tag_with_polarity_and_type': 'GENERAL-IMPRESSION-NEUTRAL-implicit',
   'type': 'label-implicit',
   'label': 'GENERAL-IMPRESSION',
   'polarity': 'NEUTRAL'}],
 'Wir würden wiederkommen.')